In [186]:
# Import files
import moviepy.editor as mp 
import wave
import numpy as np

In [187]:
word_size = 16 # Size of the number in byte format

### Encryption

In [188]:
# Load video files and change it into the wav format
video = mp.VideoFileClip("Video.mp4") 
audio = video.audio
audio.write_audiofile("Audio.wav")
audio = wave.open("Audio.wav", mode='rb')

chunk:   0%|                                                                        | 0/2986 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in Audio.wav


MoviePy - Done.


In [189]:
# Load bytes from the audio file
frame_data = audio.readframes(audio.getnframes())
audio_bytes = bytearray(list(frame_data))

In [190]:
# Frame data to be written first element is number of frame object
frame_data = [4,165,175,200,1000]

In [191]:
# Convert the frame into bit data of word size and then store it in the audio file
frame_data = ''.join(list(map(lambda number: '{0:0{width}b}'.format(number,width=word_size),frame_data)))
frame_data = list(map(int,frame_data))

In [192]:
# Peform steganography encrpt the message into the the audio and generate modified bytes
for element, frame_bit in enumerate(frame_data):
    audio_bytes[element] = (audio_bytes[element] & 254) | frame_bit
modified_frames = bytes(audio_bytes)

In [193]:
# Store the final audio file
final_audio = wave.open('Encrypted.wav', 'wb')
final_audio.setparams(audio.getparams())
final_audio.writeframes(modified_frames)

### Decryption

In [194]:
# Load the final audio file and extract frames from it
audio = wave.open("Encrypted.wav",'rb')
frame_data = audio.readframes(audio.getnframes())
frame_bytes = bytearray(list(frame_data))

In [195]:
# Extract bits from the message
extract_message_bits = [frame_bytes[i] & 1 for i in range(len(frame_bytes))]

In [196]:
# Extract number of frames which is the first value in the message
number_elements = int(str("".join(list(map(str,extract_message_bits[:word_size])))),2)

In [197]:
# Extract number of frames which is the first value in the message
elements = extract_message_bits[word_size:(number_elements+1)*word_size]
frame_list = list(np.array(elements).reshape(number_elements,word_size))
number = list(map(lambda x: int("".join(list(map(str,x))),2),frame_list))

In [198]:
# Final frame data
print(number)

[165, 175, 200, 1000]
